In [1]:

from functools import partial
import nltk

import torch
from context_enforcement.models.context_enforcer import compute_context_boundary
from context_enforcement.trainers.train_bart import model_init
from context_enforcement.data.common import create_text_tokenizer, SmartCollator
from context_enforcement.trainers.common import get_dataset_specified_tasks
from pytorch_lightning import seed_everything

import sys
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "2"
seed_everything(1376)

Global seed set to 1376
[nltk_data] Downloading package punkt to /home/nlplab/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
Global seed set to 1376


1376

In [9]:
z=torch.ones((1,2),dtype=torch.long,device='cuda')
z.device

device(type='cuda', index=0)

In [10]:
torch.ones_like(z).device

device(type='cuda', index=0)

In [2]:
configs = {}
configs["model_base"] = "facebook/bart-base"
configs["task_type"] = "xsum"

In [3]:
tokenizer = create_text_tokenizer(configs["model_base"])

task_dataset_gen = get_dataset_specified_tasks(configs["task_type"])

train_dataset = None
eval_dataset = None
test_dataset = None
if task_dataset_gen is not None:
    raw_dataset = task_dataset_gen(tokenizer=tokenizer, )
    train_dataset = raw_dataset['train']
    eval_dataset = raw_dataset['validation']
    test_dataset = raw_dataset['test']

model_builder = model_init(
        vocab_size=len(train_dataset.tokenizer),
        model_base=configs["model_base"],
    )

Found cached dataset xsum (/home/nlplab/.cache/huggingface/datasets/xsum/default/1.2.0/082863bf4754ee058a5b6f6525d0cb2b18eadb62c7b370b095d1364050a52b71)


  0%|          | 0/3 [00:00<?, ?it/s]

In [40]:
generator = model_builder()
train_model_path = "trained_models_sum_boundary/bart_base_model_context_enforcer/checkpoint-12753/pytorch_model.bin"
state_dict = torch.load(train_model_path)
generator.load_state_dict(state_dict)
device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")

Some weights of BartForContextualRecovery were not initialized from the model checkpoint at facebook/bart-base and are newly initialized: ['encoder.layers.0.context_enforcer._wrc.bias', 'encoder.layers.1.context_enforcer_layer_norm.bias', 'encoder.layers.5.context_enforcer._left_mha.in_proj_bias', 'encoder.layers.2.context_enforcer._wlc.bias', 'encoder.layers.3.context_enforcer._left_mha.out_proj.bias', 'encoder.layers.4.context_enforcer._wlc.bias', 'encoder.layers.1.context_enforcer._left_mha.in_proj_weight', 'encoder.layers.0.context_enforcer._right_mha.in_proj_weight', 'encoder.layers.2.context_enforcer._left_mha.in_proj_bias', 'encoder.layers.4.context_enforcer._wlc.weight', 'encoder.layers.0.context_enforcer_layer_norm.weight', 'encoder.layers.2.context_enforcer._left_mha.out_proj.bias', 'encoder.layers.5.context_enforcer_layer_norm.weight', 'encoder.layers.5.context_enforcer._right_mha.out_proj.weight', 'encoder.layers.3.context_enforcer_layer_norm.bias', 'encoder.layers.2.contex

In [47]:
te1= test_dataset[6]
b_input_ids = te1.input_ids.view(1, -1).to(device)
b_input_mask = te1.attention_mask.view(1, -1).to(device)

In [48]:
seed_everything(1376)
boundary_sample =  (0.1, 0.45)
seq_len = te1.input_ids[:1024].shape[0]

boundary_start = int(0.45*seq_len) 
context_boundary = (boundary_start,boundary_start+220)#compute_context_boundary(seq_len,boundary_sample,200)
context_boundary

Global seed set to 1376


(460, 680)

In [63]:
generator.eval()
with torch.no_grad():
    bb=generator.generate(input_ids=b_input_ids[:,:1024],
                attention_mask=b_input_mask[:,:1024],
                context_boundary=(150,400),
                eos_token_id=test_dataset.tokenizer.eos_token_id,
        max_length=129,
        early_stopping=True,
        use_cache=True,
        num_beams=5,
        )
test_dataset.tokenizer.batch_decode(bb,clean_up_tokenization_spaces=True,skip_special_tokens=True)

['Olympic champion Wendy Houvenaghel has accused British Cycling of "systematically discriminating" against her.']

In [53]:
test_dataset.tokenizer.decode(te1.labels,clean_up_tokenization_spaces=True,skip_special_tokens=True)

'A "medal at any cost" approach created a "culture of fear" at British Cycling, says former rider Wendy Houvenaghel.'

In [51]:
te1.input_ids.shape

torch.Size([1187])

In [58]:
print(test_dataset.tokenizer.decode(te1.input_ids,clean_up_tokenization_spaces=True,skip_special_tokens=True))

Media playback is not supported on this device
The Olympic silver medallist accused the organisation of "ageism" and having "zero regard" for her welfare.
She is the latest high-profile cyclist to come forward after Jess Varnish, Nicole Cooke and Emma Pooley criticised the World Class programme.
Houvenaghel told the BBC she felt "vindicated" by a leaked draft report detailing British Cycling's failures.
The report said British Cycling "sanitised" its own investigation into claims former technical director Shane Sutton used sexist language towards Varnish, who went public last April about her treatment.
British Cycling subsequently admitted it did not pay "sufficient care and attention" to the wellbeing of staff and athletes at the expense of winning medals, an approach Houvenaghel attested to in her BBC interview.
Both Sutton and predecessor Sir Dave Brailsford have now left British Cycling.
Houvenaghel, 42, spoke to BBC Sport during its State of Sport week, which on Thursday examines 

In [31]:
boundary_mask

(360, 610)

In [36]:
from torch.utils.data import DataLoader,SequentialSampler
import tqdm
test_data_loader = DataLoader(test_dataset,batch_size=12,
                              sampler= SequentialSampler(test_dataset),
                              collate_fn= SmartCollator(
            pad_token_id=train_dataset.tokenizer.pad_token_id, max_len=800,
            context_max_len=110,
            context_sampling_bounds=(0.1, 0.45)
        )
                              )

output_summaries =[]
for batch in tqdm.tqdm(test_data_loader):
    b_input_ids = batch['input_ids'].to(device)
    b_input_mask = batch['attention_mask'].to(device)
    
    seq_len = b_input_ids.shape[1]

    boundary_start = int(0.45*seq_len) 
    context_boundary = (boundary_start, boundary_start+110)
    
    boundary_mask =  batch.get("boundary",)
    bb=generator.generate(input_ids=b_input_ids,
            attention_mask=b_input_mask,
            context_boundary=boundary_mask,
            num_beams=10,
            do_sample=False,
            num_return_sequences=1,
            max_length=140)
    sentences = test_dataset.tokenizer.batch_decode(bb,clean_up_tokenization_spaces=True,skip_special_tokens=True)
    output_summaries+=sentences

100%|██████████| 945/945 [08:14<00:00,  1.91it/s]


In [65]:
#context_enforcement/data/common.py
from context_enforcement.data.common import write_to_file
write_to_file(output_summaries[:len(test_dataset)], "outputs/attempt1/simple_2")

Done


In [10]:
targets = [tokenizer.decode(c.labels,clean_up_tokenization_spaces=True,skip_special_tokens=True) for c in test_dataset]

In [37]:
targets[690],output_summaries2[690],output_summaries[690]

('Musician Koffi Olomide has been taken into custody in the Democratic Republic of Congo, days after he was deported from Kenya for allegedly kicking one of his dancers at an airport in Nairobi.',
 'Omar Olomide has been arrested in the Democratic Republic of Congo on suspicion of assaulting a group of dancers.',
 'Omar Olomombe has been detained by police in the Democratic Republic of Congo for allegedly assaulting a group of dancers.')

In [14]:
import evaluate
metrics = evaluate.combine(['bleu','meteor',"rouge"])


[nltk_data] Downloading package wordnet to /home/nlplab/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package punkt to /home/nlplab/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package omw-1.4 to /home/nlplab/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


In [38]:
scores2 = metrics.compute(predictions=output_summaries,references=targets)

In [29]:
scores = metrics.compute(predictions=output_summaries2,references=targets)

In [30]:
scores

{'bleu': 0.08190144303338862,
 'precisions': [0.39916558741021463,
  0.13484576712224944,
  0.06490478666102122,
  0.03461400195153127],
 'brevity_penalty': 0.7810193894484071,
 'length_ratio': 0.8018247587808374,
 'translation_length': 209249,
 'reference_length': 260966,
 'meteor': 0.28558238733460617,
 'rouge1': 0.34475899874216664,
 'rouge2': 0.13444853158452552,
 'rougeL': 0.279360220097055,
 'rougeLsum': 0.27932587220115546}

In [39]:
scores2

{'bleu': 0.08004077243766287,
 'precisions': [0.39668147524032016,
  0.1326978952590178,
  0.06327462141553002,
  0.03340461786535229],
 'brevity_penalty': 0.7793378458842136,
 'length_ratio': 0.8004414368155238,
 'translation_length': 208888,
 'reference_length': 260966,
 'meteor': 0.28278544337874784,
 'rouge1': 0.34230456443341767,
 'rouge2': 0.13258619227287083,
 'rougeL': 0.27739509890876,
 'rougeLsum': 0.2774115466752346}